Crop images

In [ ]:
import os
import json
from PIL import Image

# Paths
images_path = "../A-Dataset-and-Benchmark-for-Malaria-Life-Cycle-Classification-in-Thin-Blood-Smear-Images/IML_Malaria"
annotation_path = "../A-Dataset-and-Benchmark-for-Malaria-Life-Cycle-Classification-in-Thin-Blood-Smear-Images/annotations.json"
output_dir = "cell_dataset/"

# Lag mapper for hver klasse
classes = ["ring", "trophozoite", "schizont", "gametocyte", "red blood cell"]
for cls in classes:
    os.makedirs(os.path.join(output_dir, cls), exist_ok=True)

# Last inn annotasjonene
with open(annotation_path) as f:
    ground_truth = json.load(f)

# Iterer over alle bilder og beskjær celler
for entry in ground_truth:
    image_name = entry["image_name"]
    image_path = os.path.join(images_path, image_name)
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Bilde ikke funnet: {image_path}")
        continue

    for i, obj in enumerate(entry["objects"]):
        label = obj["type"]  
        if label not in classes:
            print(f"Ukjent klasse: {label} — hopper over")
            continue

        # Bounding box
        x = int(obj["bbox"]["x"])
        y = int(obj["bbox"]["y"])
        w = int(obj["bbox"]["w"])
        h = int(obj["bbox"]["h"])

        # Beskjær og lagre
        cropped = image.crop((x, y, x + w, y + h))
        save_path = os.path.join(output_dir, label, f"{image_name[:-4]}_{i}.png")
        cropped.save(save_path)

print(" Done with cropping and saving images.")


Nåværende arbeidsmappe: /Users/andreakristiane/MalariaClassification/notebooks
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
Ukjent klasse: difficult — hopper over
✅ Ferdig med beskjæring og lagring!
